In [6]:
import spacy
import numpy as np
import nltk
import re

In [7]:
nlp = spacy.load('es')

MemoryError: Error assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytesError assigning 16777216 bytes

In [ ]:
with open("pbaspacy.txt", "r") as f:
    corpus = f.read()

In [ ]:
# Tokenizing
tokenizer = nltk.data.load('tokenizers/punkt/spanish.pickle')
raw = tokenizer.tokenize(corpus)

In [ ]:
print(raw)

In [ ]:
#limpiando
clean = []
for sent in raw:
    a = re.sub(r'&#\w+;', '', sent)
    b = re.sub(r'[^\w]', ' ', a)
    c = b.rstrip().lstrip()
    clean.append(c)

In [ ]:
for i,a in enumerate(clean[:27]):
    print(str(i)+")", a)

In [ ]:
#Para sacar las letras que quedaron colgadas al principio
last_clean = []
for a in clean:
    aux = a.split()
    if len(aux[0]) == 1:
        if aux[0] not in {'A', 'Y', 'O'}:
            del aux[0]
    last_clean.append(aux)

In [ ]:
for i,a in enumerate(last_clean[:5]):
    print(str(i)+")", a)

In [ ]:
#flatten list
flat = []
for s in last_clean:
    for w in s:
        flat.append(w)

In [ ]:
#Aprovechamos que tenemos una lista de palabras para poder contar su ocurrencia
count = dict()
for w in flat:
    if w not in count:
        count[w] = 1
    else:
        count[w] += 1

In [ ]:
#cuantas veces aparece la palabra "el" ? ---> 8
count["Rodolfo"]

In [ ]:
#Ahora volvemos a unir las palabras a la oración que pertenecen
final = []
for i,s in enumerate(last_clean):
    a = ' '.join(last_clean[i])
    final.append(a)

In [ ]:
for i,a in enumerate(final[:27]):
    print(str(i)+")", a)

In [ ]:
#Pasamos las oraciones pre-procesadas a Spacy
parsed_sents = []
for doc in nlp.pipe(final, batch_size=10000, n_threads=4):
    parsed_sents.append(doc)

In [ ]:
parsed_sents[0:5]

In [ ]:
#diccionario con las triplas de cada palabra
dicc_trip = dict()
for s in parsed_sents:
    for w in s:
        tags = [(w.text, w.dep_, w.head.text)]
        if str(w) not in dicc_trip:
            dicc_trip[str(w)] = tags
        else:
            dicc_trip[str(w)] += (tags)

In [ ]:
parsed_sents[1]

In [ ]:
#lista de listas con tuplas de la forma (palabra, Pos)
pos = []
for s in parsed_sents:
    aux=[]
    for w in s:
        aux.append((w, w.pos_))
    pos.append(aux)

In [ ]:
pos[2]

In [ ]:
#todas las triplas relacionadas con "el"
dicc_trip["el"]

In [ ]:
for key, value in dicc_trip.items():
    aux = []
    for fst,snd,trd in value:
        a = snd+'.'+trd
        aux.append(a)
        dicc_trip[key] = aux

In [ ]:
dicc_trip["el"]

In [ ]:
# Lista de etiquetas para cada oración
tag_sents = [[(token.text,token.tag_,token.pos_) for token in parsedEx] for parsedEx in parsed_sents]

# Lista de triplas de dependencias para cada oración
#dep_trips = [[(token.orth_, token.dep_, token.head.orth_) for token in parsedEx] for parsedEx in parsed_sents]



In [ ]:
# final ---> lista de oraciones procesadas
# tag_sents ---> tags de cada oración
# dep_trips ---> triplas de dep de cada oración

In [ ]:
# Juntamos todo
#triplas = [tri for tri in zip(final, tag_sents, dep_trips)]

#ahora tenemos algo de la forma (oración, tags, tripla de dependencia)

In [ ]:
tag_sents[:2]

In [ ]:
tags_clean = []
for i,a in enumerate(tag_sents):
    for fst,snd,trd in a:
        aux = snd.split("|")
        tgs = []
        for x in aux:
            y = x.split("=")
            if len(y) == 1:
                tgs.append(y[0])
            else:
                tgs.append((y[0], y[1]))
        tags.append((fst, tgs, trd))

In [ ]:
clean_info_word = []
for a in tag_sents:
    for fst,snd,trd in a:
        aux = snd.split("|")
        clean_info_word.append((fst,aux,trd))

In [ ]:
#Ahora tenemos una lista de triplas, donde el primer elem es la palabra, el segundo sus tags y el tercero su PoS
print(clean_info_word[0])

In [ ]:
pos[2:4]

In [ ]:
from collections import defaultdict
dicc_forward = defaultdict(list)
dicc_backward = defaultdict(list)
for tup in pos:
    for j,(fst,snd) in enumerate(tup):
        if j != 0:
            dicc_backward[fst.text].append((tup[j-1][0].text, tup[j-1][1]))
        else:
            dicc_backward[fst.text].append(("[W.Start]", "[T.Start]"))
        if j != len(tup)-1:
            dicc_forward[fst.text].append((tup[j+1][0].text, tup[j+1][1]))
        else:
            dicc_forward[fst.text].append(("[W.End]", "[T.End]"))

In [ ]:
dicc_backward["Clara"]

In [ ]:
clean_info_word[0]

In [ ]:
dicc_backward["Clara"]

In [ ]:
#Creando el diccionario
dicc = dict()
#features = dict()
for fst,snd,trd in clean_info_word:
    features = defaultdict(int)
    pos = 'PoS__' + trd
    if fst not in dicc and count[fst] > 2:      #si la palabra es nueva y ocurre mas de n veces -> la agrego
        dicc[fst.lower()] = features
        features[pos] = 1
        #features['LowerCase'] = str(fst.islower)
        for i in snd:
            features[i] = 1
        for i in dicc_trip[fst]:                 #diccionario de triplas
            features[i] = 1
        for i in dicc_backward[fst]:
            features[i[0]+"-1"] += 1
            features[i[1]+"-1"] += 1
        for i in dicc_forward[fst]:
            features[i[0]+"+1"] += 1
            features[i[0]+"+1"] += 1
            
            
    elif fst in dicc:                        #la palabra ya se encuentra -> actualizo el dicc
        has_it = dicc[fst]
        if pos in has_it:
            has_it[pos] += 1
        else:
            has_it[pos] = 1
        for tag in snd:
            if tag in has_it:
                has_it[tag] += 1
            else:
                has_it[tag] = 1
        

In [ ]:
dicc_backward["Clara"]

In [ ]:
dicc_forward["Clara"]

In [ ]:
count["Clara"]

In [ ]:
dicc['a']

In [ ]:
#Ahora preparamos la lista una lista de diccs para vectorizar
#Tambien vamos a guardar las palabras que vamos a vectorizar para poder recuperarlas mas tarde
lista_para_vectorizar = []
check_list = []
for key, value in dicc.items():
    check_list.append(key)
    lista_para_vectorizar.append(value)

In [ ]:
#Corroboramos la correspondencia
for i,w in enumerate(check_list):
    a = lista_para_vectorizar[i]
    #print(w + " -->", a)

In [ ]:
#Vectorizado
from sklearn.feature_extraction import DictVectorizer
v = DictVectorizer(sparse=False)
X = v.fit_transform(lista_para_vectorizar)

In [ ]:
X

In [ ]:
from sklearn.decomposition import TruncatedSVD

#Reducción de dimensionalidad
svd = TruncatedSVD(n_components=47, n_iter=7, random_state=42)
svd.fit_transform(X)

In [ ]:
#Normalización
import sklearn
Y = sklearn.preprocessing.normalize(X, axis=0)

In [ ]:
Y

In [ ]:
#Clustering
from sklearn.cluster import KMeans
km = KMeans(n_clusters=3)
km.fit(Y)

In [ ]:
labels = km.predict(Y)

In [ ]:
print(labels)

In [ ]:
#Clusters obtenidos
clusters = defaultdict(set)
for i, label in enumerate(labels):
    clusters[label].add(check_list[i])
print(clusters)

In [ ]:
import pickle

file_name = "dict_cluster"
with open(file_name, 'wb') as f:
    pickle.dump(clusters, f)